# Spooky Author Identification


In [ ]:
#Import Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

In [ ]:
# Reading Training, Testing dataset
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")


In [ ]:
train.head(10)

In [ ]:
# Number of records in Training dataset is 19579 records
train.shape[0]

In [ ]:
# Lets check how the records are distributed against the authors in training dataset
sns.countplot(x="author", data=train, palette = "Greens_d")

In [ ]:
#Lets create 3 different dataframes according to the Authors
train_eap = train[train.author=='EAP']
train_hpl = train[train.author=='HPL']
train_mws = train[train.author=='MWS']

In [ ]:
#Wordcloud for EAP
from wordcloud import WordCloud, STOPWORDS
Alltext= (' '.join(train_eap['text']))
wc = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(Alltext)

plt.figure(figsize=(15,5));
plt.imshow(wc);
plt.axis('off');
plt.title('Word Cloud for Edgar Allan Poe');

In [ ]:
#Wordcloud for MWS
Alltext= (' '.join(train_mws['text']))
wc = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(Alltext)

plt.figure(figsize=(15,5));
plt.imshow(wc);
plt.axis('off');
plt.title('Word Cloud for Mary Shelley');

In [ ]:
#Wordcloud for HPL
Alltext= (' '.join(train_hpl['text']))
wc = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(Alltext)

plt.figure(figsize=(15,5));
plt.imshow(wc);
plt.axis('off');
plt.title('Word Cloud for HP Lovecraft');

In [ ]:
# lets create some features from the text columns on Train dataset
train["length"] = train["text"].apply(lambda x: len(str(x).split()))
test["length"] = test["text"].apply(lambda x: len(str(x).split()))

In [ ]:
print(train.groupby(by=['author'])['length'].mean())
# the mean length of text for each author is almost same for HPL and MWS.

In [ ]:
# Lets start extracting features from NLTK packages
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
eng_stopwords = set(stopwords.words("english"))
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus_text = []
#train.shape[0]
for i in range(0, train.shape[0]):
    corpus = train["text"][i]
    corpus = corpus.lower()
    corpus = corpus.split()
    ps = PorterStemmer()
    corpus = [ps.stem(word) for word in corpus if not word in eng_stopwords]
    corpus = ' '.join(corpus)
    corpus_text.append(corpus)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)

In [ ]:
X = cv.fit_transform(corpus_text).toarray()
y = train.iloc[:, 2].values

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# Lets use navies bayes classified (MultinomialNB)
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
ACC

Since we have decent accuracy let me try to create the prediction on test output file.

In [ ]:
# Run the NLTK on Test dataset to create the corpus
test_corpus_text = []
for i in range(0, test.shape[0]):
    corpus = test["text"][i]
    corpus = corpus.lower()
    corpus = corpus.split()
    ps = PorterStemmer()
    corpus = [ps.stem(word) for word in corpus if not word in eng_stopwords]
    corpus = ' '.join(corpus)
    test_corpus_text.append(corpus)

In [ ]:
X_test_output = cv.transform(test_corpus_text).toarray()

In [ ]:
y_prob_output = classifier.predict_proba(X_test_output)

In [ ]:
y_prob_output

In [ ]:
submission_df = pd.DataFrame(y_prob_output,index=test['id'],columns=['EAP','HPL','MWS'])

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv")

Let me try decision tree using the same data and compare the accuracy

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 1007)

In [ ]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
ACC

It looks like Multilogistic reg algorithm  is doing much better than the random forest, stay tuned, i would be adding more classifier like SVM, Decision tree and xgboost. Please upvote to encourage!!